In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path
import hvplot.pandas

#read in all files
jobs_2022 = Path('Resources_2/2022_jobsdata.csv')
jobs_2021 = Path('Resources_2/2021_jobsdata.csv')
jobs_df_1 = pd.DataFrame(pd.read_csv(jobs_2022))
jobs_df2 = pd.DataFrame(pd.read_csv(jobs_2021))




In [7]:
#filter for total employee numbers for zip code 
jobs = jobs_df_1[['Label (Grouping)','Civilian employed population 16 years and over']]
jobs_df = pd.DataFrame(jobs)
jobs_df.head(33)

#create empty lists for getting the zip data and the total employment data
zips = []

totals = []

#this for loops looks for specific index to find the zip code and adds to the empty established list
for i in range(0, len(jobs_df), 16):
    new_zip = jobs_df.iloc[i,0]
    zips.append(new_zip)

#this for loop looks for specific index to find the total employment number change and adds tot he empty established list
for i in range(2, len(jobs_df), 16):
    new_total = jobs_df.iloc[i,1]
    totals.append(new_total)

zips
totals

#turn the zips and totals data into a full dataframe
full_df = pd.DataFrame({'Zip Code': zips,
                        'Total Employed 2022': totals})
full_df

#clean up zip code data
full_df['Zip Code'] = full_df['Zip Code'].str.replace('ZCTA5 ', '')
full_df

#clean up employed data and turn the data into integer for calculation
full_df['Total Employed 2022'] = full_df['Total Employed 2022'].str.replace(',','')
full_df['Total Employed 2022'] = full_df['Total Employed 2022'].astype(int)

full_df


,Zip Code,Total Employed 2022
0,73949,714
1,73960,0
2,75001,11749
3,75002,36013
4,75006,27304
...,...,...
1987,79934,12137
1988,79935,8527
1989,79936,50837
1990,79938,38761


In [8]:
jobs2 = jobs_df2[['Label (Grouping)','Civilian employed population 16 years and over']]
jobs_df2 = pd.DataFrame(jobs2)
jobs_df2.head(33)

zips = []

totals = []

for i in range(0, len(jobs_df2), 16):
    new_zip = jobs_df2.iloc[i,0]
    zips.append(new_zip)

for i in range(2, len(jobs_df2), 16):
    new_total = jobs_df2.iloc[i,1]
    totals.append(new_total)

zips
totals

full_df2 = pd.DataFrame({'Zip Code': zips,
                        'Total Employed 2021': totals})
full_df2

full_df2['Zip Code'] = full_df2['Zip Code'].str.replace('ZCTA5 ', '')
full_df2

full_df2['Total Employed 2021'] = full_df2['Total Employed 2021'].str.replace(',','')
full_df2['Total Employed 2021'] = full_df2['Total Employed 2021'].astype(int)

full_df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Zip Code             1992 non-null   object
 1   Total Employed 2021  1992 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 23.5+ KB


In [9]:
comp_df = pd.merge(full_df, full_df2,on='Zip Code')
change = comp_df['Total Employed 2022'] / comp_df['Total Employed 2021'] - 1

comp_df['Employment Change 2022'] = (change*100).map('{:.2f}%'.format)
comp_df

top25 = pd.read_csv('Resources_2/top_25.csv')
top25_df = pd.DataFrame(top25)


comp_df.rename(columns={"Zip Code": "Zip"}, inplace=True)

comp_df['Zip'] = comp_df['Zip'].astype(int)

new_df = pd.merge(top25_df,comp_df,on='Zip',how='inner')

new_df = new_df.drop(columns=['Total Employed 2021'])
new_df = new_df.drop(columns=['Total Employed 2022'])

new_df




,Zip,City,County,Latitude,Longitude,2018,2019,2020,2021,2022,Employment Change 2022
0,76519,Burlington,Milam,30.98978,-97.03579,-12.25,21.45,-28.13,-28.57,93.20,58.59%
1,79766,Odessa,Ector,31.71014,-102.36185,4.54,12.90,-0.23,40.11,9.92,16.03%
2,75414,Bells,Grayson,33.62504,-96.43193,-9.58,18.65,-0.64,-11.93,9.87,10.18%
3,77304,Conroe,Montgomery,30.33097,-95.51538,7.29,8.38,7.58,2.25,9.86,7.50%
4,76853,Lometa,Lampasas,31.24774,-98.39047,-1.97,-5.70,-8.45,11.93,9.85,9.11%
5,77564,Hull,Liberty,30.21811,-94.65989,-15.51,12.51,-18.86,13.38,9.79,2.94%
6,76377,Petrolia,Clay,34.01010,-98.24191,-3.29,-30.01,-25.23,-3.86,9.77,4.70%
7,75438,Dodd City,Fannin,33.60701,-96.07462,-1.09,9.25,-1.92,-2.06,9.76,8.46%
8,79119,Amarillo,Randall,35.10876,-102.01968,3.21,6.89,5.63,10.14,9.73,9.20%
9,78950,New Ulm,Austin,29.90735,-96.48518,-10.75,1.40,2.93,16.26,9.70,5.74%


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (Zip,Employment Change 2022)